In [1]:
import pandas as pd
from os import listdir
from datetime import datetime, timedelta

In [2]:
#Parsear String a timestamp
def parse_time(x):
    datetime_object = datetime.strptime(x,'%H:%M:%S')
    return datetime_object 

#Obtener los indices y el numero de columnas faltantes
def get_index(df):
  news_rows = []
  for row in df.itertuples(index=True):
    index = row.Index
    date_1 = parse_time(df['Timestamp'][index])
    if index < (len(df)-1):
      date_2 =  parse_time( df['Timestamp'][index+1])
    else:
      date_2 =  parse_time( df['Timestamp'][index])
    minutes = ((date_2 - date_1).seconds)/60
    if minutes > 1:
      news_rows.append({'index': index, 'num': minutes-1})
  return news_rows

#obtener el nombre de los archivos
def ls(pwd): 
    return listdir(pwd)

#Funcion que agrega las filas faltante y crea un nuevo DF
def create_df(df, news_rows):
  indexs = []
  num_rows = []
  for key in news_rows:
    if key['num'] < 1050:
      indexs.append(key['index'])
      num_rows.append(key['num'])
    else:
        continue

  nf = pd.DataFrame(columns=['Date', 'Timestamp', 'Ticker', 'OpenPrice',
                             'HighPrice', 'LowPrice', 'ClosePrice', 'TotalVolume', 
                             'TotalQuantity', 'TotalTradeCount'])
  index = 0
  for row in df.itertuples(index=True):
    lista= [row.Date, row.Timestamp, row.Ticker,
            row.OpenPrice, row.HighPrice, row.LowPrice, row.ClosePrice,
            row.TotalVolume, row.TotalQuantity, row.TotalTradeCount]
    
    nf.loc[index] = lista
    
    if row.Index in indexs:
        index_array = indexs.index(row.Index)
        index = index + 1
        
        key = 0
        while key < num_rows[index_array]:
          nf.loc[index] = [row.Date, (parse_time(row.Timestamp) + timedelta(minutes=key+1)).strftime('%H:%M:%S'),
                         row.Ticker,
                            row.OpenPrice, row.HighPrice, row.LowPrice, row.ClosePrice,
                           0,0,0]
          key = key + 1
          index = index + 1
            
    index = index + 1
  #nf.to_csv('2018-complete.csv', encoding='utf-8', index=False)
  return nf

#ef limpiar_datos(tickers):
#   for ticker in tickers:
#       file = pd.read_csv('./emisoras/2018/'+ticker)
#       df = pd.DataFrame(file)
#       date = df[df.columns[1]]
#       new_df = df[(date>'09:30:00')&(date<'16:01:00')]
        #new_df.to_csv('./emisoras/2018-limpio/'+ticker, encoding='utf-8', index=False)

def completar_datos(tickers):
    for ticker in tickers:
        file = pd.read_csv('./emisoras/2018-limpio/'+ticker)
        df = pd.DataFrame(file)    
        index = get_index(df)
        new_df = create_df(df, index)
        new_df.to_csv('./emisoras/2018-limpio-1/'+ticker, encoding='utf-8', index=False)

In [3]:
def limpiar_datos(tickers):
    first = []
    for ticker in tickers:
        file = pd.read_csv('./emisoras/2018/'+ticker)
        df = pd.DataFrame(file)
        date = df[df.columns[1]]
        new_df = df[(date>='09:30:00')&(date<'16:01:00')]
        new_df.to_csv('./emisoras/2018-limpio/'+ticker, encoding='utf-8', index=False)
        
        file = pd.read_csv('./emisoras/2018-limpio/'+ticker)
        df_clean = pd.DataFrame(file)
        nf = df_clean[0:1]
        if nf['Timestamp'][0] == '09:30:00':
            continue
        else:
            
            stock = { 'Date':   nf[nf.columns[0]][0],          'Timestamp': '09:30:00',
                  'Ticker': nf[nf.columns[2]][0],          'OpenPrice': nf[nf.columns[3]][0], 
                  'HighPrice': nf[nf.columns[4]][0],        'LowPrice': nf[nf.columns[5]].min(),
                  'ClosePrice': nf[nf.columns[6]][0], 'TotalVolume': 0,
                  'TotalQuantity': 0,    'TotalTradeCount': 0 }
        
            df_stocks = pd.DataFrame([stock],columns=['Date',          'Timestamp',     'Ticker',     'OpenPrice',
                                             'HighPrice',     'LowPrice',      'ClosePrice', 'TotalVolume', 
                                             'TotalQuantity', 'TotalTradeCount'])
        
        
            nf_final = df_stocks.append(new_df,ignore_index=True)
    
        #oc = new_df[0:1]['Timestamp'].values[0]
        #print (oc == "09:30:00")
        
            nf_final.to_csv('./emisoras/2018-limpio/'+ticker, encoding='utf-8', index=False)

In [4]:
#tickers =  ls('./emisoras/2018/') #Limpiamos los datos en el tiempo de la bolsa
tickers = ['BHR.csv']
limpiar_datos(tickers)

In [5]:
#tickers_2 = ls('./emisoras/2018-limpio/')
completar_datos(tickers)

In [6]:

oc = (parse_time('09:31:00') + timedelta(minutes=1)).strftime('%H:%M:%S')

In [7]:
oc

'09:32:00'